## <font size=6> Title about atmospheric rivers and pollution</font>
<br><br>
<font size=4>
**Julia Asplund** (julia.asplund@aces.su.se), in collaboration with Lea Haberstock <p>
Date, 2022<p>
eScience Tools in Climate Science: Linking Observations with Modelling <p>
Group assistant, fearless leader, person we would not have managed without: Remy Lapere 
</font>

## Abstract

## Introduction

-Mention differences between arctic and antarctic in terms of pollution

## Methods

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import s3fs
import intake
import functions_JA as fun
#import os
#import warnings
#from matplotlib import rc,animation
#from matplotlib.animation import FuncAnimation
#from IPython import display
#import cartopy as cy
#import cartopy.crs as ccrs

### Preprocessing

Describe preprocessing and AR mask. Link to file. 

### Load data 

Loading AR mask, pre calculated as described:

In [ ]:
ar_mask = xr.open_dataset('combined_hist_q94.nc')

Fetching AOD data from met.no:

In [ ]:
#Define connection to met.no bucket
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", 
                       secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", 
                       client_kwargs=dict(endpoint_url="https://rgw.met.no"))

#path to aod data file, in Ada's folder
s3path_aod = 's3://escience2022/Ada/monthly/od550aer_AERday_NorESM2-LM_historical_r1i1p1f1_gn_2000101-20141231.nc'

#Importing file and dropping unused parameters
aod_ds = xr.open_dataset(s3.open(s3path_aod), drop_variables =['time_bnds','lat_bnds','lon_bnds'])

Loading variables to be considered, from pangeo:

In [ ]:
#Years considered, corresponding to AR mask
start_year='2000'
end_year='2015'

#Define url and connection
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

#Search database for the right model, experiment and variables. Save catalog as dictionary, and finally as xarray dataset
cat = col.search(source_id=['NorESM2-LM'], experiment_id=['historical'], table_id=['day'], variable_id=['clt','pr','tas','hus'], member_id=['r1i1p1f1'])
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})
dataset_list = list(dset_dict.keys())
dset = dset_dict[dataset_list[0]]
vars_ds = dset.sel(member_id='r1i1p1f1',time=slice(str(start_year)+"-01-01", str(end_year)+"-01-01"))

### AR masking and pollution threshold

Since I want to investigate the properties of clean versus polluted AR, I need to sort the detected AR depending pollution level. The only relevant aerosol particle parameter with daily output for this CMIP experiment is AOD, and therefore it will be used to sort the AR. Aerosol particle concentrations at the poles generally very low, so low that air transported from mid latitudes will almost always be polluted in comparison. Therefore, I define the thresholds for clean and polluted AR based the AOD at mid latitudes. AR that reach the Arctic (above 60$^{\cdot}$ North) are considered polluted if the average AOD in the river is within the upper 25th percentile of AOD for the whole time period at latitudes between 30$^{\cdot}$ and 60$^{\cdot}$ North. Arctic AR with an average AOD in the lower 25th percentile are considered clean. Antarctic AR are sorted in the same way, but depending on the AOD between 30$^{\cdot}$ and 60$^{\cdot}$ South. 

In [ ]:
#Creating a dictionary to store the pollution limits for the sorting of ARs 
aod_lim= {}
aod_lim['midlat_75th'] = aod_hist.sel(lat = slice(30,60)).od550aer.quantile(0.75,skipna=True)
aod_lim['midlat_25th'] = aod_hist.sel(lat = slice(30,60)).od550aer.quantile(0.25,skipna=True)
aod_lim['midlowlat_75th'] = aod_hist.sel(lat = slice(-60,-30)).od550aer.quantile(0.75,skipna=True)
aod_lim['midlowlat_25th'] = aod_hist.sel(lat = slice(-60,-30)).od550aer.quantile(0.25,skipna=True)

Now I can apply the AR mask to the AOD data and sort the rivers. The function `sort_ar_by_aod` takes the AOD dataset, the AR mask and the pollution limits, loops over every timestep, applies the mask and then checks the average AOD of every river to sort it. Returned is an AOD dataset with all gridboxes outside the rivers masked out, separated into polluted, clean, and intermediate pollution levels. These parameters can be used as an AR mask for each pollution category separately. There is also a count of the number of rivers in each category for every timestep.

In [ ]:
arc_aod_ar = fun.sort_ar_by_aod(aod_ds.sel(lat=slice(60,90)), ar_mask.sel(lat=slice(60,90)), aod_lim['midlat_75th'], aod_lim['midlat_25th'])
ant_aod_ar = fun.sort_ar_by_aod(aod_ds.sel(lat=slice(-90,-60)), ar_hist.sel(lat=slice(-90,-60)), aod_lim['midlowlat_75th'], aod_lim['midlowlat_25th'])

Next I have to apply the AR masks to the dataset containing the variables of interest. The masked datasets are stored in two different dictionaries, one for each pole. In order to plot histograms of the variables they need to be in 1-dimensional arrays, with all NAN-values (from the masked out data) removed. These are also stored in the dictionaries, to simplify plotting.

In [ ]:
#Create empty dictionaries to fill with variables and define variables and pollution levels considered:
arc_plotting_vars = {}
ant_plotting_vars = {}
flat_vars = ['clt','pr','tas','hus']
levs = ['clean','mid','poll']


for lev in levs:
    #Create a dictionary in each pollution level
    arc_lev_dict = {}
    ant_lev_dict = {}
    
    #Mask the dataset with the AR mask for current pollution level, separate for both ploes and store in dictionary created above.
    arc_lev_dict['ar_masked'] = vars_ds.sel(lat=slice(60,90)).where(arc_aod_ar[f'{lev}_ar_aod'].notnull() )
    ant_lev_dict['ar_masked'] = vars_ds.sel(lat=slice(-90,-60)).where(ant_aod_ar[f'{lev}_ar_aod'].notnull() )
    
    for var in flat_vars:    
        #For current variable, flatten the masked array and remove NaN values
        arc_lev_dict[f'{var}_flat']= lev_dict['ar_masked'][var].values.flatten()[~np.isnan(lev_dict['ar_masked'][var].values.flatten())]
        ant_lev_dict[f'{var}_flat']= lev_dict['ar_masked'][var].values.flatten()[~np.isnan(lev_dict['ar_masked'][var].values.flatten())]
        
        if var == 'pr':
            #Convert precipiation from kg/m^2/s to mm/day
            arc_lev_dict[f'{var}_flat']= arc_lev_dict[f'{var}_flat']*86400
            ant_lev_dict[f'{var}_flat']= ant_lev_dict[f'{var}_flat']*86400
    
    #save dictionary of data in current pollution level
    arc_plotting_vars[lev]=arc_lev_dict
    ant_plotting_vars[lev]=ant_lev_dict
    

## Results and discussion

In [ ]:
fig, axs = plt.subplots(1,len(list(plotting_vars['clean'].keys())[1:-1])+1, dpi=100, figsize=(12,3))

for i, var in enumerate(list(plotting_vars['clean'].keys())[1:-1]):
    axs[i].hist(plotting_vars['clean'][var], alpha = 0.6,bins=15, label='clean AR', weights=np.zeros_like(plotting_vars['clean'][var]) + 1. / plotting_vars['clean'][var].size)
    axs[i].hist(plotting_vars['poll'][var], alpha = 0.6,bins=15, label='polluted AR', weights=np.zeros_like(plotting_vars['poll'][var]) + 1. / plotting_vars['poll'][var].size)

sum = aod_ar_ds.sum(dim='time')

axs[3].pie([sum[count] for count in ['clean_ar_counts','mid_ar_counts','poll_ar_counts']]
           ,labels = ['Clean','Intermediate','Polluted'], colors = ['tab:blue', 'tab:green','tab:orange'], autopct='%1.f%%',shadow=True)    
    
axs[0].set_xlabel('Cloud fraction [%]')
axs[0].set_ylabel('frequency distribution')
axs[1].set_xlabel('Precipitation [mm/day]')
axs[2].set_xlabel('Surface temperature [K]')
axs[3].set_xlabel('AR distribution')

axs[0].set_yscale('log')
axs[1].set_yscale('log')
fig.suptitle('Arctic')
axs[0].legend()
plt.tight_layout()
plt.savefig('figures/arctic_aod_4panels.jpeg', dpi=100)
plt.show()

Discuss
-AR detection (supplements)
-AOD vs vertically resolved aerosol parameter (Where is pollution picked up, does aod really reflect what is carried with the AR, etc)

## Summary and conclusions

## Supplementary material

Comparicon to Jonathans product, discussion and q threshold, maps for comparison